In [4]:
"""
Processa HTMLs e extrai: partes, advogados (validação de nomes), juiz, sentença
VERSÃO FINAL - Só marca tipo se encontrar nome do advogado
"""

from bs4 import BeautifulSoup
import pandas as pd
import os
import glob
import re

def e_data(texto):
    """Verifica se o texto parece ser uma data"""
    padroes_data = [
        r'\d{2}/\d{2}/\d{4}',
        r'\d{2}-\d{2}-\d{4}',
        r'\d{4}-\d{2}-\d{2}'
    ]
    for padrao in padroes_data:
        if re.search(padrao, texto):
            return True
    return False

def e_nome_valido(texto):
    """Verifica se parece um nome válido"""
    if not texto or len(texto) < 5:
        return False
    if e_data(texto):
        return False
    texto_lower = texto.lower()
    palavras_invalidas = ['se for uma parte', 'digite a senha', 'movimentações', 'cpf', 'cnpj', 'oab']
    if any(palavra in texto_lower for palavra in palavras_invalidas):
        return False
    palavras = texto.split()
    if len(palavras) < 2:
        return False
    return True

def extrair_dados_html(caminho_html):
    """Extrai dados completos incluindo múltiplos advogados"""
    
    numero_processo = os.path.basename(caminho_html).replace('.html', '')
    
    with open(caminho_html, 'r', encoding='utf-8') as f:
        html = f.read()
    
    soup = BeautifulSoup(html, 'html.parser')
    
    dados = {
        'num_processo': numero_processo,
        'requerente': None,
        'advogado_requerente': None,
        'tipo_adv_requerente': None,
        'requerido': None,
        'advogado_requerido': None,
        'tipo_adv_requerido': None,
        'juiz': None,
        'procedente': None,
        'desc': None,
        'link_processo': f"https://esaj.tjac.jus.br/cpopg/show.do?processo.numero={numero_processo}",
        'link_pdf': None
    }
    
    texto = soup.get_text()
    linhas_texto = texto.split('\n')
    
    # Buscar Reclamante e seus advogados
    encontrou_reclamante = False
    advogados_requerente = []
    tipo_temp_requerente = None
    
    for i, linha in enumerate(linhas_texto):
        linha_strip = linha.strip()
        linha_lower = linha_strip.lower()
        
        # Procurar reclamante
        if dados['requerente'] is None:
            if any(termo in linha_lower for termo in ['reclamante', 'requerente', 'autor']):
                encontrou_reclamante = True
                for j in range(1, 10):
                    if i + j < len(linhas_texto):
                        candidato = linhas_texto[i + j].strip()
                        if candidato and len(candidato) > 3:
                            if not any(x in candidato.lower() for x in ['reclamante', 'requerente', 'autor', 'd. públic', 'defensor', 'advogad', 'cpf', 'cnpj', ':', 'soc.']):
                                if e_nome_valido(candidato):
                                    dados['requerente'] = candidato
                                    break
        
        # Procurar advogados do reclamante
        if encontrou_reclamante and not dados['advogado_requerente']:
            # Identificar o tipo
            if any(termo in linha_lower for termo in ['d. públic', 'defensor públic', 'defensora públic', 'defensoria pública']):
                tipo_temp_requerente = 'público'
                # Coletar múltiplos defensores
                for j in range(1, 15):
                    if i + j < len(linhas_texto):
                        candidato = linhas_texto[i + j].strip()
                        if any(x in candidato.lower() for x in ['reclamado', 'requerido', 'réu']):
                            break
                        if candidato and e_nome_valido(candidato):
                            if not any(x in candidato.lower() for x in ['d. públic', 'defensor', 'cpf', 'oab', ':']):
                                advogados_requerente.append(candidato)
                
                # Só marcar tipo se encontrou pelo menos um nome
                if advogados_requerente:
                    dados['advogado_requerente'] = ' | '.join(advogados_requerente)
                    dados['tipo_adv_requerente'] = tipo_temp_requerente
                    encontrou_reclamante = False
            
            elif any(termo in linha_lower for termo in ['advogad', 'soc. advogados']):
                tipo_temp_requerente = 'privado'
                for j in range(1, 15):
                    if i + j < len(linhas_texto):
                        candidato = linhas_texto[i + j].strip()
                        if any(x in candidato.lower() for x in ['reclamado', 'requerido', 'réu']):
                            break
                        if candidato and e_nome_valido(candidato):
                            if not any(x in candidato.lower() for x in ['advogad', 'soc.', 'cpf', 'oab', ':']):
                                advogados_requerente.append(candidato)
                
                # Só marcar tipo se encontrou pelo menos um nome
                if advogados_requerente:
                    dados['advogado_requerente'] = ' | '.join(advogados_requerente)
                    dados['tipo_adv_requerente'] = tipo_temp_requerente
                    encontrou_reclamante = False
    
    # Buscar Reclamado e seus advogados
    encontrou_reclamado = False
    advogados_requerido = []
    tipo_temp_requerido = None
    
    for i, linha in enumerate(linhas_texto):
        linha_strip = linha.strip()
        linha_lower = linha_strip.lower()
        
        # Procurar reclamado
        if dados['requerido'] is None:
            if any(termo in linha_lower for termo in ['reclamado', 'requerido', 'réu']):
                encontrou_reclamado = True
                for j in range(1, 10):
                    if i + j < len(linhas_texto):
                        candidato = linhas_texto[i + j].strip()
                        if candidato and len(candidato) > 3:
                            if not any(x in candidato.lower() for x in ['reclamado', 'requerido', 'réu', 'soc. advogados', 'advogad', 'cpf', 'cnpj', ':']):
                                if e_nome_valido(candidato):
                                    dados['requerido'] = candidato
                                    break
        
        # Procurar advogados do reclamado
        if encontrou_reclamado and not dados['advogado_requerido']:
            if any(termo in linha_lower for termo in ['d. públic', 'defensor públic', 'defensora públic', 'defensoria pública']):
                tipo_temp_requerido = 'público'
                for j in range(1, 15):
                    if i + j < len(linhas_texto):
                        candidato = linhas_texto[i + j].strip()
                        if any(x in candidato.lower() for x in ['movimentações', 'data', 'movimento']):
                            break
                        if candidato and e_nome_valido(candidato):
                            if not any(x in candidato.lower() for x in ['d. públic', 'defensor', 'cpf', 'oab', ':']):
                                advogados_requerido.append(candidato)
                
                # Só marcar tipo se encontrou nome
                if advogados_requerido:
                    dados['advogado_requerido'] = ' | '.join(advogados_requerido)
                    dados['tipo_adv_requerido'] = tipo_temp_requerido
                    encontrou_reclamado = False
            
            elif any(termo in linha_lower for termo in ['advogad', 'soc. advogados']):
                tipo_temp_requerido = 'privado'
                for j in range(1, 15):
                    if i + j < len(linhas_texto):
                        candidato = linhas_texto[i + j].strip()
                        if any(x in candidato.lower() for x in ['movimentações', 'data', 'movimento']):
                            break
                        if candidato and e_nome_valido(candidato):
                            if not any(x in candidato.lower() for x in ['advogad', 'soc.', 'cpf', 'oab', ':']):
                                advogados_requerido.append(candidato)
                
                # Só marcar tipo se encontrou nome
                if advogados_requerido:
                    dados['advogado_requerido'] = ' | '.join(advogados_requerido)
                    dados['tipo_adv_requerido'] = tipo_temp_requerido
                    encontrou_reclamado = False
    
    # Extrair juiz
    for i, linha in enumerate(linhas_texto):
        if 'juiz' in linha.lower() and i + 1 < len(linhas_texto):
            proximo = linhas_texto[i + 1].strip()
            if proximo and len(proximo) > 5 and not proximo.startswith('('):
                dados['juiz'] = proximo
                break
    
    # Termos para sentença
    termos_parcial = [
        'julgo procedente em parte', 'julgo procedentes em parte',
        'julgado procedente em parte', 'julgados procedentes em parte',
        'julgo parcialmente procedente', 'julgado parcialmente procedente'
    ]
    
    termos_improcedente = [
        'julgo improcedente', 'julgo improcedentes',
        'julgado improcedente', 'julgados improcedentes'
    ]
    
    termos_procedente = [
        'julgo procedente', 'julgo procedentes',
        'julgado procedente', 'julgados procedentes'
    ]
    
    # Buscar sentença
    linhas_tabela = soup.find_all('tr')
    
    for linha in linhas_tabela:
        texto_linha = linha.get_text()
        texto_linha_lower = texto_linha.lower()
        texto_limpo = ' '.join(texto_linha.split())
        
        resultado_encontrado = None
        
        if any(termo in texto_linha_lower for termo in termos_parcial):
            resultado_encontrado = 2
        elif any(termo in texto_linha_lower for termo in termos_improcedente):
            resultado_encontrado = 0
        elif any(termo in texto_linha_lower for termo in termos_procedente):
            if not any(termo in texto_linha_lower for termo in termos_improcedente):
                resultado_encontrado = 1
        
        if resultado_encontrado is not None:
            link = linha.find('a', href=True)
            
            if link or dados['procedente'] is None:
                dados['procedente'] = resultado_encontrado
                dados['desc'] = texto_limpo
                
                if link:
                    dados['link_pdf'] = 'https://esaj.tjac.jus.br' + link['href']
                    break
    
    return dados


# ==================== PROCESSAR TODOS OS HTMLs ====================

print("="*60)
print("PROCESSANDO HTMLs - VERSÃO FINAL")
print("="*60 + "\n")

arquivos_html = sorted(glob.glob('arquivos/*.html'))
total = len(arquivos_html)

print(f"Total de HTMLs: {total}\n")

resultados = []

for i, caminho in enumerate(arquivos_html):
    if (i + 1) % 100 == 0:
        print(f"Processando... {i+1}/{total}")
    
    dados = extrair_dados_html(caminho)
    resultados.append(dados)

# Criar DataFrame
df = pd.DataFrame(resultados)

# Limpar campos
for coluna in ['requerente', 'advogado_requerente', 'requerido', 'advogado_requerido', 'juiz', 'desc']:
    if coluna in df.columns:
        df[coluna] = df[coluna].str.replace('\n', ' ', regex=False)
        df[coluna] = df[coluna].str.replace('\r', ' ', regex=False)
        df[coluna] = df[coluna].str.replace(r'\s+', ' ', regex=True)
        df[coluna] = df[coluna].str.strip()

# Salvar CSV
df.to_csv('processos_tjac_completo.csv', index=False, encoding='utf-8-sig')

# Resumo
print("\n" + "="*60)
print("RESUMO FINAL")
print("="*60)
print(f"Total: {len(df)}")
print(f"Requerente: {df['requerente'].notna().sum()} ({(df['requerente'].notna().sum()/len(df)*100):.1f}%)")
print(f"  Advogado: {df['advogado_requerente'].notna().sum()} ({(df['advogado_requerente'].notna().sum()/len(df)*100):.1f}%)")
print(f"    • Público: {(df['tipo_adv_requerente'] == 'público').sum()}")
print(f"    • Privado: {(df['tipo_adv_requerente'] == 'privado').sum()}")
print(f"    • Sem tipo: {df['tipo_adv_requerente'].isna().sum()}")
print(f"Requerido: {df['requerido'].notna().sum()} ({(df['requerido'].notna().sum()/len(df)*100):.1f}%)")
print(f"  Advogado: {df['advogado_requerido'].notna().sum()} ({(df['advogado_requerido'].notna().sum()/len(df)*100):.1f}%)")
print(f"    • Público: {(df['tipo_adv_requerido'] == 'público').sum()}")
print(f"    • Privado: {(df['tipo_adv_requerido'] == 'privado').sum()}")
print(f"    • Sem tipo: {df['tipo_adv_requerido'].isna().sum()}")
print(f"Juiz: {df['juiz'].notna().sum()} ({(df['juiz'].notna().sum()/len(df)*100):.1f}%)")
print(f"Sentença: {df['procedente'].notna().sum()} ({(df['procedente'].notna().sum()/len(df)*100):.1f}%)")
print(f"  • Procedentes (1): {(df['procedente'] == 1).sum()}")
print(f"  • Improcedentes (0): {(df['procedente'] == 0).sum()}")
print(f"  • Parciais (2): {(df['procedente'] == 2).sum()}")
print(f"\n✓ CSV: processos_tjac_completo.csv")
print("="*60)

print("\nPrimeiras 5 linhas:")
print(df[['num_processo', 'requerente', 'advogado_requerente', 'tipo_adv_requerente']].head())

PROCESSANDO HTMLs - VERSÃO FINAL

Total de HTMLs: 1200

Processando... 100/1200
Processando... 200/1200
Processando... 300/1200
Processando... 400/1200
Processando... 500/1200
Processando... 600/1200
Processando... 700/1200
Processando... 800/1200
Processando... 900/1200
Processando... 1000/1200
Processando... 1100/1200
Processando... 1200/1200

RESUMO FINAL
Total: 1200
Requerente: 1191 (99.2%)
  Advogado: 956 (79.7%)
    • Público: 142
    • Privado: 814
    • Sem tipo: 244
Requerido: 1014 (84.5%)
  Advogado: 877 (73.1%)
    • Público: 51
    • Privado: 826
    • Sem tipo: 323
Juiz: 1199 (99.9%)
Sentença: 756 (63.0%)
  • Procedentes (1): 166
  • Improcedentes (0): 300
  • Parciais (2): 290

✓ CSV: processos_tjac_completo.csv

Primeiras 5 linhas:
           num_processo                                     requerente  \
0  00000011520238010021                 Maria do Rosário Souza Moreira   
1  00000026220258010010                         Oseias Barros da Silva   
2  000000502202580100